In [1]:
!git clone https://github.com/PonyGE/PonyGE2.git

fatal: destination path 'PonyGE2' already exists and is not an empty directory.


In [2]:
!cd PonyGE2/ && git pull

Already up to date.


In [3]:
import shutil
import os
from tqdm.auto import tqdm
import logging
import time
import sys
import subprocess

logger = logging.getLogger(__name__)

experiment = "regression"

def run_experiment(experiment_num=None):
    """Executa um experimento GP usando o Python do ambiente virtual correto"""
    logger.info(f"{'='*60}")
    if experiment_num is not None:
        logger.info(f"Starting experiment {experiment_num}")
    else:
        logger.info("Starting experiment")
    logger.info(f"{'='*60}")
    
    # Verificar se o arquivo já existe, se não, copiar
    fitness_path = f"./PonyGE2/src/fitness/{experiment}.py"
    if not os.path.exists(fitness_path):
        logger.info(f"Copying fitness function: ./fitness/{experiment}.py -> {fitness_path}")
        shutil.copy(f"./fitness/{experiment}.py", fitness_path)
    else:
        logger.info(f"Fitness function already exists at {fitness_path}")
    
    # Usar o Python do ambiente virtual atual
    python_exec = sys.executable
    logger.info(f"Using Python: {python_exec}")
    
    # Medir tempo de execução do GP
    start_time = time.time()
    logger.info("Running PonyGE2 evolution (this may take a while)...")
    
    # Executar PonyGE2 com subprocess para garantir que usa o Python correto
    try:
        result = subprocess.run(
            [python_exec, 'ponyge.py', '--parameters', f'../../parameters/{experiment}.txt'],
            cwd='./PonyGE2/src',
            capture_output=True,
            text=True,
            timeout=3600  # 1 hora timeout
        )
        captured = result.stdout.split('\n')
        if result.stderr:
            logger.warning(f"PonyGE2 stderr: {result.stderr[:500]}")
    except Exception as e:
        logger.error(f"Error running PonyGE2: {e}")
        captured = [str(e)]
    
    gp_runtime = time.time() - start_time
    
    logger.info(f"Experiment completed in {gp_runtime:.2f} seconds")
    return captured, gp_runtime

/Users/lucasreis/Documents/RegressorGE/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch
import torch.nn.functional as F
import numpy as np
from sewar.full_ref import mse
from openpyxl import load_workbook
import logging
import time
import psutil
import os
from functools import wraps

logger = logging.getLogger(__name__)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logger.info(f"Using device: {device}")

# Funções para cálculo de métricas adicionais
def calculate_rmse(y_true, y_pred):
    """Calcula Root Mean Squared Error"""
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def calculate_r2(y_true, y_pred):
    """Calcula R² (Coefficient of Determination)"""
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    if ss_tot == 0:
        return 0.0
    return 1 - (ss_res / ss_tot)

def get_memory_usage():
    """Retorna uso de memória em MB"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024  # Convert to MB

def timing_and_memory(func):
    """Decorator para medir tempo e memória"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        start_memory = get_memory_usage()
        result = func(*args, **kwargs)
        end_time = time.time()
        end_memory = get_memory_usage()
        execution_time = end_time - start_time
        memory_used = end_memory - start_memory
        return result, execution_time, memory_used
    return wrapper

def extremeVal(reference, val):
    if (val > 1e2*reference) or (val < -1e2*reference):
        return 0
    else:
        return val

split_type_index = {"arima": 2, "sarima": 1}
models_values_column = {
    "arima": {
        "series": 1,
        "arima": 3,
        "mlp": 4,
        "svr": 5,
        "rbf": 6,
    },
    "sarima": {
        "series": 6,
        "sarima": 2,
        "mlp": 3,
        "svr": 4,
        "rbf": 5,
    },
}

def load_data(dataset_name, dataset_path, linear_type):
    logger.info(f"Loading dataset: {dataset_name} (type: {linear_type}) from {dataset_path}")
    wb = load_workbook(dataset_path)
    dataset_name_arr = [dataset_name]

    dataset = {}
    for name in dataset_name_arr:
        logger.debug(f"Processing sheet: {name}")
        ws = wb[name]
        columns = [ws["A"], ws["B"], ws["C"],
                    ws["D"], ws["E"], ws["F"], ws["G"]]

        sheet = [[] for _ in range(len(columns[0]))]

        for c in columns:
            for index, item in enumerate(c):
                sheet[index].append(item.value)

        sheet = np.array(sheet)
        dataset[name] = {"raw": sheet}

    for name, _ in dataset.items():
        sheet = dataset[name]["raw"]

        dataset[name]["series"] = {
            "treino": [], "teste": [], "validacao": []}
        dataset[name][linear_type] = {
            "treino": [], "teste": [], "validacao": []}
        dataset[name]["mlp"] = {"treino": [], "teste": [], "validacao": []}
        dataset[name]["svr"] = {"treino": [], "teste": [], "validacao": []}
        dataset[name]["rbf"] = {"treino": [], "teste": [], "validacao": []}

        logger.debug(f"Processing {len(sheet)} rows")
        for row in sheet[1:]:
            split_type = row[split_type_index[linear_type]]
            if linear_type == "arima" and row[4] == None:
                continue
            key = None
            if split_type.lower() == "teste":
                key = "teste"
            elif split_type.lower() == "validacao" or split_type.lower() == "validação":
                key = "validacao"
            elif split_type.lower() == "treinamento":
                key = "treino"
            else:
                continue

            for model_key, val in models_values_column[linear_type].items():
                dataset[name][model_key][key].append(float(row[val]))

        for type_data in ["teste", "treino", "validacao"]:
            dataset[name]["series"][type_data] = np.array(
                dataset[name]["series"][type_data])
            dataset[name][linear_type][type_data] = np.array(
                dataset[name][linear_type][type_data])
            dataset[name]["mlp"][type_data] = np.array(
                dataset[name]["mlp"][type_data])
            dataset[name]["svr"][type_data] = np.array(
                dataset[name]["svr"][type_data])
            dataset[name]["rbf"][type_data] = np.array(
                dataset[name]["rbf"][type_data])

    fixExtreme = np.vectorize(extremeVal)

    for name in dataset.keys():
        reference = np.max(dataset[name]["series"]["treino"])

        for type_data in ["teste", "treino", "validacao"]:
            dataset[name]["mlp"][type_data] = fixExtreme(
                reference, dataset[name]["mlp"][type_data])
            dataset[name]["svr"][type_data] = fixExtreme(
                reference, dataset[name]["svr"][type_data])
            dataset[name]["rbf"][type_data] = fixExtreme(
                reference, dataset[name]["rbf"][type_data])
            dataset[name][linear_type][type_data] = fixExtreme(
                reference, dataset[name][linear_type][type_data])

    logger.info(f"Dataset loaded successfully. Train: {len(dataset[name]['series']['treino'])}, "
                f"Val: {len(dataset[name]['series']['validacao'])}, "
                f"Test: {len(dataset[name]['series']['teste'])} samples")
    return dataset[dataset_name]


def build_model(phenotype):
    logger.debug(f"Building model from phenotype: {phenotype[:100]}...")
    model = {"window_size": 0, "linear": {}, "nlinear": {}}

    window_size, model_phenotype = phenotype.split('~')
    model["window_size"] = int(window_size)

    linear, nlinear = model_phenotype.split(';')

    weight_tuple, linear_model = linear.split(':')

    weight_values = weight_tuple.removeprefix('(').removesuffix(')')
    model["linear"][linear_model] = [
        float(x) for x in weight_values.split(' ')]

    nlinear_parts = nlinear.split(',')

    for i in range(len(nlinear_parts)):
        weight_tuple, nlinear_model = nlinear_parts[i].split(':')
        weight_values = weight_tuple.removeprefix('(').removesuffix(')')

        if nlinear_model in model["nlinear"]:
            model["nlinear"][nlinear_model] += [
                float(x) for x in weight_values.split(' ')]
        else:
            model["nlinear"][nlinear_model] = [
                float(x) for x in weight_values.split(' ')]

    logger.debug(f"Model built: window_size={model['window_size']}, "
                f"linear={list(model['linear'].keys())}, "
                f"nlinear={list(model['nlinear'].keys())}")
    return model

def predict_metrics(model, arima_dataset, sarima_dataset, split):
    """Calcula múltiplas métricas: MSE, RMSE, R²"""
    if "arima" in model["linear"].keys():
        dataset = arima_dataset
        linear_type = "arima"
    else:
        dataset = sarima_dataset
        linear_type = "sarima"

    window_size = model["window_size"]
    y_true = dataset["series"][split]
    X_train = apply_window(
        window_size, split, linear_type, dataset, model_names=model["nlinear"].keys())
    kernel = []
    for i in range(window_size):
        for _, val in model["linear"].items():
            kernel.append(val[i])
        for _, val in model["nlinear"].items():
            kernel.append(val[i])
    y_pred = create_prediction(X_train, np.reshape(kernel, (1, -1)))
    
    # Calcular todas as métricas
    mse_value = mse(y_true, y_pred)
    rmse_value = calculate_rmse(y_true, y_pred)
    r2_value = calculate_r2(y_true, y_pred)
    
    logger.debug(f"MSE on {split}: {mse_value:.6e}, RMSE: {rmse_value:.6e}, R²: {r2_value:.6f}")
    
    return {
        'mse': mse_value,
        'rmse': rmse_value,
        'r2': r2_value
    }

def predict_mse(model, arima_dataset, sarima_dataset, split):
    """Mantida para compatibilidade - retorna apenas MSE"""
    metrics = predict_metrics(model, arima_dataset, sarima_dataset, split)
    return metrics['mse']

def apply_window(window_size, data_type, linear_type, dataset, model_names=None):
    size_pred = len(dataset["series"][data_type])
    new_data = []
    for w in range(window_size):
        linear = np.concatenate(
            [np.zeros(w), dataset[linear_type][data_type]])
        linear = linear[:size_pred]
        row = np.array(linear)
        for model_name in model_names:
            nonlinear = np.concatenate(
                [np.zeros(w), dataset[model_name][data_type]])
            nonlinear = nonlinear[:size_pred]
            row = np.column_stack([row, nonlinear])
        if len(new_data) == 0:
            new_data = row
        else:
            new_data = np.column_stack([new_data, row])
    return np.array(new_data)

def create_prediction(X, kernel):
    X = torch.tensor(np.expand_dims(X, axis=(0, 1))).float()
    kernel = torch.tensor(np.expand_dims(kernel, axis=(0, 1))).float()
    X.to(device)
    kernel.to(device)
    result = F.conv2d(X, kernel)
    return result.numpy().squeeze().astype("f")

def save_results_to_csv(results_data, filename='results/experiment_results.csv'):
    """Salva resultados em CSV"""
    import pandas as pd
    import os
    
    # Criar diretório se não existir
    os.makedirs(os.path.dirname(filename) if os.path.dirname(filename) else '.', exist_ok=True)
    
    # Converter para DataFrame
    df = pd.DataFrame([results_data])
    
    # Se o arquivo existe, adicionar ao final, senão criar novo
    if os.path.exists(filename):
        df.to_csv(filename, mode='a', header=False, index=False)
    else:
        df.to_csv(filename, mode='w', header=True, index=False)
    
    logger.info(f"Results saved to {filename}")

In [5]:
from tqdm.auto import tqdm
import logging
import re
import pandas as pd
import glob

logger = logging.getLogger(__name__)

def treat_experiment_return(result, experiment_num=None, gp_runtime=None, verbose=False):
    """Processa resultado do experimento e salva métricas, tempo e memória em CSV"""
    
    dataset_name = ''
    phenotype = ''
    
    # Handle result - it might be a list of strings or a single string
    if isinstance(result, list):
        result_lines = result
    else:
        result_lines = str(result).split('\n')
    
    # Debug: log what we're parsing
    logger.info(f"Parsing {len(result_lines)} lines from PonyGE2 output")
    
    # Parse the output - look for phenotype in multiple formats
    for i, line in enumerate(result_lines):
        # Convert to string if needed
        line_str = str(line).strip()
        
        # Look for phenotype - try multiple patterns
        if not phenotype:
            # Pattern 1: "Phenotype: <value>"
            if "Phenotype:" in line_str:
                parts = line_str.split("Phenotype:", 1)
                if len(parts) > 1:
                    phenotype = parts[1].strip()
                    logger.info(f"Found phenotype (pattern 1) at line {i}: {phenotype[:100]}...")
            
            # Pattern 2: Next line after "Phenotype:" header
            if i > 0 and "Phenotype:" in str(result_lines[i-1]).strip() and line_str and not phenotype:
                phenotype = line_str
                logger.info(f"Found phenotype (pattern 2) at line {i}: {phenotype[:100]}...")
        
        # Look for dataset name
        if not dataset_name and "DATASET_NAME" in line_str:
            # Try different formats
            if "DATASET_NAME: " in line_str:
                dataset_name = line_str.split("DATASET_NAME: ", 1)[1].strip()
            elif "DATASET_NAME:" in line_str:
                dataset_name = line_str.split("DATASET_NAME:", 1)[1].strip()
            logger.info(f"Found dataset_name at line {i}: {dataset_name}")
    
    # Debug: print last few lines if phenotype not found
    if not phenotype:
        logger.warning("Phenotype not found in output. Last 10 lines:")
        for i, line in enumerate(result_lines[-10:], start=len(result_lines)-10):
            logger.warning(f"Line {i}: {str(line)[:200]}")
    
    # Fallback 1: Try to read from PonyGE2 output file (best.txt)
    if not phenotype:
        logger.warning("Phenotype not found in output, trying to read from PonyGE2 output file...")
        try:
            # PonyGE2 typically saves to a file in the output directory
            import glob
            best_files = glob.glob('./PonyGE2/src/**/best.txt', recursive=True)
            if not best_files:
                best_files = glob.glob('./PonyGE2/**/best.txt', recursive=True)
            
            if best_files:
                # Get the most recent file
                best_file = max(best_files, key=os.path.getmtime)
                logger.info(f"Reading phenotype from {best_file}")
                with open(best_file, 'r') as f:
                    content = f.read()
                    # Look for "Phenotype:" in the file
                    for line in content.split('\n'):
                        if 'Phenotype:' in line:
                            parts = line.split('Phenotype:', 1)
                            if len(parts) > 1:
                                phenotype = parts[1].strip()
                                logger.info(f"Found phenotype in file: {phenotype[:100]}...")
                                break
                        # Or check if next line after "Phenotype:" header
                        if 'Phenotype:' in line and not phenotype:
                            # Get next non-empty line
                            lines = content.split('\n')
                            idx = lines.index(line)
                            for next_line in lines[idx+1:]:
                                if next_line.strip():
                                    phenotype = next_line.strip()
                                    logger.info(f"Found phenotype in file (next line): {phenotype[:100]}...")
                                    break
                            if phenotype:
                                break
        except Exception as e:
            logger.error(f"Could not read from PonyGE2 output file: {e}")
    
    # Fallback 2: read from parameter file if dataset name not found
    if not dataset_name:
        logger.warning("Dataset name not found in output, reading from parameter file...")
        try:
            with open('./parameters/regression.txt', 'r') as f:
                for line in f:
                    if line.startswith('DATASET_NAME'):
                        dataset_name = line.split(':', 1)[1].strip()
                        logger.info(f"Read dataset name from parameter file: {dataset_name}")
                        break
        except Exception as e:
            logger.error(f"Could not read parameter file: {e}")
    
    # Validate we have what we need
    if not dataset_name:
        raise ValueError("Could not determine dataset name from PonyGE2 output or parameter file")
    
    if not phenotype:
        # Last resort: raise error with helpful message
        error_msg = "Could not find phenotype in PonyGE2 output. "
        error_msg += "Please check if PonyGE2 completed successfully and printed the phenotype."
        error_msg += f"\nTotal lines parsed: {len(result_lines)}"
        raise ValueError(error_msg)
    
    if verbose:
        print(f'Dataset: {dataset_name}')
        print(f'Fenótipo: {phenotype}')

    # Medir tempo e memória para avaliação
    start_time = time.time()
    start_memory = get_memory_usage()
    
    logger.info(f"Loading datasets for evaluation...")
    arima_dataset = load_data(dataset_name, './datasets/arima_data.xlsx', 'arima')
    sarima_dataset = load_data(dataset_name, './datasets/sarima_data.xlsx', 'sarima')
    
    logger.info("Building model from phenotype...")
    model = build_model(phenotype)
    
    logger.info("Evaluating model on all splits...")
    train_metrics = predict_metrics(model, arima_dataset, sarima_dataset, 'treino')
    val_metrics = predict_metrics(model, arima_dataset, sarima_dataset, 'validacao')
    test_metrics = predict_metrics(model, arima_dataset, sarima_dataset, 'teste')
    
    eval_time = time.time() - start_time
    eval_memory = get_memory_usage() - start_memory

    if verbose:
        print('Métricas Treino - MSE:', f"{train_metrics['mse']:.3e}, RMSE: {train_metrics['rmse']:.3e}, R²: {train_metrics['r2']:.6f}")
        print('Métricas Validação - MSE:', f"{val_metrics['mse']:.3e}, RMSE: {val_metrics['rmse']:.3e}, R²: {val_metrics['r2']:.6f}")
        print('Métricas Teste - MSE:', f"{test_metrics['mse']:.3e}, RMSE: {test_metrics['rmse']:.3e}, R²: {test_metrics['r2']:.6f}")

    # Preparar dados para CSV
    results_data = {
        'experiment_num': experiment_num,
        'dataset': dataset_name,
        'phenotype': phenotype,
        'mse_train': train_metrics['mse'],
        'rmse_train': train_metrics['rmse'],
        'r2_train': train_metrics['r2'],
        'mse_val': val_metrics['mse'],
        'rmse_val': val_metrics['rmse'],
        'r2_val': val_metrics['r2'],
        'mse_test': test_metrics['mse'],
        'rmse_test': test_metrics['rmse'],
        'r2_test': test_metrics['r2'],
        'gp_runtime_seconds': gp_runtime if gp_runtime else None,
        'eval_time_seconds': eval_time,
        'eval_memory_mb': eval_memory,
        'total_memory_mb': get_memory_usage()
    }
    
    # Salvar em CSV
    save_results_to_csv(results_data, 'results/experiment_results.csv')
    
    return phenotype, test_metrics['mse'], results_data

In [6]:
# Executar múltiplos experimentos GP para TODOS os datasets
import shutil

verbose = True
num_experiments = 15
all_datasets = ['CARSALES', 'Electricity', 'GAS', 'LAKEERIE', 'Nordic', 'PIGS', 'POLLUTION', 'REDWINE', 'SUNSPOT', 'B1H']

logger.info(f"Starting GP experiments on {len(all_datasets)} datasets, {num_experiments} runs each")
logger.info("="*60)

all_results = []

for dataset_name in tqdm(all_datasets, desc="Datasets", unit="dataset"):
    logger.info(f"\n{'#'*60}")
    logger.info(f"DATASET: {dataset_name}")
    logger.info(f"{'#'*60}")
    
    # Atualizar o arquivo de parâmetros com o dataset atual
    param_file = './parameters/regression.txt'
    
    # Ler o arquivo
    with open(param_file, 'r') as f:
        lines = f.readlines()
    
    # Atualizar a linha do DATASET_NAME
    with open(param_file, 'w') as f:
        for line in lines:
            if line.startswith('DATASET_NAME:'):
                f.write(f'DATASET_NAME:           {dataset_name}\n')
            else:
                f.write(line)
    
    logger.info(f"Updated parameter file to use dataset: {dataset_name}")
    
    # Executar experimentos para este dataset
    dataset_results = []
    
    for i in tqdm(range(num_experiments), desc=f"GP Runs ({dataset_name})", unit="run", leave=False):
        logger.info(f"\n{'='*60}")
        logger.info(f"Dataset: {dataset_name} - Experiment {i+1}/{num_experiments}")
        logger.info(f"{'='*60}")
        
        # Executar experimento GP (retorna resultado e tempo)
        result, gp_runtime = run_experiment(experiment_num=i+1)
        
        # Processar resultado e salvar em CSV
        phenotype, test_mse, results_data = treat_experiment_return(
            result, 
            experiment_num=i+1, 
            gp_runtime=gp_runtime,
            verbose=verbose
        )
        
        dataset_results.append((phenotype, test_mse, results_data))
        all_results.append((dataset_name, phenotype, test_mse, results_data))

        if verbose:
            print()
            print('---------------------')
            print()
    
    # Resumo para este dataset
    fitnesses = [x[1] for x in dataset_results]
    logger.info(f"\n{'='*60}")
    logger.info(f"RESULTS FOR {dataset_name}")
    logger.info(f"{'='*60}")
    logger.info(f'Average MSE: {np.average(fitnesses):.6e}')
    logger.info(f'Std. dev. MSE: {np.std(fitnesses):.6e}')
    logger.info(f'Best fitness: {np.min(fitnesses):.6e}')
    logger.info(f"{'='*60}\n")

logger.info(f"\n{'#'*60}")
logger.info("ALL GP EXPERIMENTS COMPLETED!")
logger.info(f"{'#'*60}")

# Resumo final por dataset
print('\n' + '='*60)
print('FINAL SUMMARY - ALL DATASETS')
print('='*60)

for dataset_name in all_datasets:
    dataset_fitnesses = [x[2] for x in all_results if x[0] == dataset_name]
    if dataset_fitnesses:
        print(f'\n{dataset_name}:')
        print(f'  Average MSE: {np.average(dataset_fitnesses):.6e}')
        print(f'  Std. dev.: {np.std(dataset_fitnesses):.6e}')
        print(f'  Best: {np.min(dataset_fitnesses):.6e}')

print('='*60)

# Carregar e mostrar resumo do CSV
import pandas as pd
try:
    df_results = pd.read_csv('results/experiment_results.csv')
    print(f"\n✅ Total experiments saved: {len(df_results)}")
    print(f"📊 By dataset:")
    for dataset in all_datasets:
        count = len(df_results[df_results['dataset'] == dataset])
        if count > 0:
            avg_mse = df_results[df_results['dataset'] == dataset]['mse_test'].mean()
            print(f"   {dataset}: {count} runs, avg MSE = {avg_mse:.6e}")
except Exception as e:
    logger.warning(f"Could not load results CSV: {e}")

Datasets:   0%|          | 0/10 [00:00<?, ?dataset/s]PonyGE2 stderr: /Users/lucasreis/Documents/RegressorGE/.venv/lib/python3.12/site-packages/sewar/utils.py:18: UserWarning: Supplied images have different dtypes float32 and float64
  warnings.warn(msg)



Dataset: CARSALES
Fenótipo: 1~(00.99999 96.09601 96.73276 68.47294 52.20248 61.24676 50.06582 79.97892 42.01005 81.97892):sarima;(01.00533 19.47264 09.67322 16.84729 85.22024 96.32467 85.00658 47.99789 94.20100 38.19789):mlp


/Users/lucasreis/Documents/RegressorGE/.venv/lib/python3.12/site-packages/sewar/utils.py:18: UserWarning: Supplied images have different dtypes float64 and float32
  warnings.warn(msg)


Métricas Treino - MSE: 7.257e-04, RMSE: 2.694e-02, R²: 0.980346
Métricas Validação - MSE: 3.609e-02, RMSE: 1.900e-01, R²: -0.032794
Métricas Teste - MSE: 7.827e-02, RMSE: 2.798e-01, R²: -1.218037

---------------------



PonyGE2 stderr: /Users/lucasreis/Documents/RegressorGE/.venv/lib/python3.12/site-packages/sewar/utils.py:18: UserWarning: Supplied images have different dtypes float32 and float64
  warnings.warn(msg)



...

Métricas Treino - MSE: 4.255e-04, RMSE: 2.063e-02, R²: 0.989182
Métricas Validação - MSE: 2.374e-04, RMSE: 1.541e-02, R²: 0.990382
Métricas Teste - MSE: 4.370e-04, RMSE: 2.090e-02, R²: 0.985944

---------------------



PonyGE2 stderr: /Users/lucasreis/Documents/RegressorGE/.venv/lib/python3.12/site-packages/sewar/utils.py:18: UserWarning: Supplied images have different dtypes float32 and float64
  warnings.warn(msg)



Dataset: B1H
Fenótipo: 1~(00.99984 48.04842 93.16100 78.31603 24.53755 69.70194 00.94109 92.81723 12.01589 12.30720):sarima;(01.06999 35.24537 59.64644 57.60235 87.57644 82.63579 51.23789 50.51310 08.40617 12.31047):svr


/Users/lucasreis/Documents/RegressorGE/.venv/lib/python3.12/site-packages/sewar/utils.py:18: UserWarning: Supplied images have different dtypes float64 and float32
  warnings.warn(msg)
Datasets: 100%|██████████| 10/10 [1:02:07<00:00, 372.73s/dataset]

Métricas Treino - MSE: 4.255e-04, RMSE: 2.063e-02, R²: 0.989182
Métricas Validação - MSE: 2.373e-04, RMSE: 1.540e-02, R²: 0.990388
Métricas Teste - MSE: 4.368e-04, RMSE: 2.090e-02, R²: 0.985952

---------------------


FINAL SUMMARY - ALL DATASETS

CARSALES:
  Average MSE: 7.847541e-02
  Std. dev.: 5.739261e-04
  Best: 7.757194e-02

Electricity:
  Average MSE: 3.739773e-03
  Std. dev.: 6.055724e-03
  Best: 2.120223e-03

GAS:
  Average MSE: 8.743771e-03
  Std. dev.: 8.992605e-03
  Best: 3.926239e-03

LAKEERIE:
  Average MSE: 1.447301e-03
  Std. dev.: 2.689424e-05
  Best: 1.420813e-03

Nordic:
  Average MSE: 4.127348e-04
  Std. dev.: 5.334802e-04
  Best: 2.289658e-04

PIGS:
  Average MSE: 1.002233e-02
  Std. dev.: 1.813405e-04
  Best: 9.840427e-03

POLLUTION:
  Average MSE: 1.334948e-02
  Std. dev.: 3.274008e-03
  Best: 1.094033e-02

REDWINE:
  Average MSE: 1.047514e-02
  Std. dev.: 1.312779e-03
  Best: 9.794848e-03

SUNSPOT:
  Average MSE: 2.527020e-02
  Std. dev.: 1.269618e-02
  Best: 

In [7]:
from tqdm.auto import tqdm
import logging

logger = logging.getLogger(__name__)

def run_tests(dataset_name):
    """Executa testes de baseline e salva resultados em CSV"""
    linear_models = ['arima', 'sarima']
    nlinear_models = ['mlp', 'svr', 'rbf']

    logger.info(f"\n{'='*60}")
    logger.info(f"Testing baseline models on dataset: {dataset_name}")
    logger.info(f"{'='*60}")

    for linear_model in tqdm(linear_models, desc=f"Linear models ({dataset_name})", leave=False):
        phenotype = f'1~(01.00000 00.00000 00.00000 00.00000 00.00000 00.00000 00.00000 00.00000 00.00000 00.00000):{linear_model};(00.00000 00.00000 00.00000 00.00000 00.00000 00.00000 00.00000 00.00000 00.00000 00.00000):{nlinear_models[0]}'

        print(f'Dataset: {dataset_name}')
        print(f'{linear_model}')

        # Medir tempo e memória
        start_time = time.time()
        start_memory = get_memory_usage()
        
        logger.debug(f"Loading datasets for {linear_model}...")
        arima_dataset = load_data(
            dataset_name, './datasets/arima_data.xlsx', 'arima')
        sarima_dataset = load_data(
            dataset_name, './datasets/sarima_data.xlsx', 'sarima')
        model = build_model(phenotype)

        # Calcular métricas
        train_metrics = predict_metrics(model, arima_dataset, sarima_dataset, 'treino')
        val_metrics = predict_metrics(model, arima_dataset, sarima_dataset, 'validacao')
        test_metrics = predict_metrics(model, arima_dataset, sarima_dataset, 'teste')
        
        eval_time = time.time() - start_time
        eval_memory = get_memory_usage() - start_memory

        print('Métricas Treino - MSE:', f"{train_metrics['mse']:.3e}, RMSE: {train_metrics['rmse']:.3e}, R²: {train_metrics['r2']:.6f}")
        print('Métricas Validação - MSE:', f"{val_metrics['mse']:.3e}, RMSE: {val_metrics['rmse']:.3e}, R²: {val_metrics['r2']:.6f}")
        print('Métricas Teste - MSE:', f"{test_metrics['mse']:.3e}, RMSE: {test_metrics['rmse']:.3e}, R²: {test_metrics['r2']:.6f}")
        print()
        print('------------------')
        print()

        # Salvar em CSV
        results_data = {
            'experiment_num': None,
            'dataset': dataset_name,
            'method': linear_model,
            'phenotype': phenotype,
            'mse_train': train_metrics['mse'],
            'rmse_train': train_metrics['rmse'],
            'r2_train': train_metrics['r2'],
            'mse_val': val_metrics['mse'],
            'rmse_val': val_metrics['rmse'],
            'r2_val': val_metrics['r2'],
            'mse_test': test_metrics['mse'],
            'rmse_test': test_metrics['rmse'],
            'r2_test': test_metrics['r2'],
            'gp_runtime_seconds': None,
            'eval_time_seconds': eval_time,
            'eval_memory_mb': eval_memory,
            'total_memory_mb': get_memory_usage()
        }
        save_results_to_csv(results_data, 'results/baseline_results.csv')

        for nlinear_model in tqdm(nlinear_models, desc=f"Nonlinear models ({linear_model})", leave=False):
            phenotype = f'1~(01.00000 00.00000 00.00000 00.00000 00.00000 00.00000 00.00000 00.00000 00.00000 00.00000):{linear_model};(01.00000 00.00000 00.00000 00.00000 00.00000 00.00000 00.00000 00.00000 00.00000 00.00000):{nlinear_model}'

            print(f'Dataset: {dataset_name}')
            print(f'{linear_model} + {nlinear_model}')

            # Medir tempo e memória
            start_time = time.time()
            start_memory = get_memory_usage()

            arima_dataset = load_data(dataset_name, './datasets/arima_data.xlsx', 'arima')
            sarima_dataset = load_data(dataset_name, './datasets/sarima_data.xlsx', 'sarima')
            model = build_model(phenotype)

            # Calcular métricas
            train_metrics = predict_metrics(model, arima_dataset, sarima_dataset, 'treino')
            val_metrics = predict_metrics(model, arima_dataset, sarima_dataset, 'validacao')
            test_metrics = predict_metrics(model, arima_dataset, sarima_dataset, 'teste')
            
            eval_time = time.time() - start_time
            eval_memory = get_memory_usage() - start_memory

            print('Métricas Treino - MSE:', f"{train_metrics['mse']:.3e}, RMSE: {train_metrics['rmse']:.3e}, R²: {train_metrics['r2']:.6f}")
            print('Métricas Validação - MSE:', f"{val_metrics['mse']:.3e}, RMSE: {val_metrics['rmse']:.3e}, R²: {val_metrics['r2']:.6f}")
            print('Métricas Teste - MSE:', f"{test_metrics['mse']:.3e}, RMSE: {test_metrics['rmse']:.3e}, R²: {test_metrics['r2']:.6f}")
            print()
            print('------------------')
            print()

            # Salvar em CSV
            results_data = {
                'experiment_num': None,
                'dataset': dataset_name,
                'method': f'{linear_model} + {nlinear_model}',
                'phenotype': phenotype,
                'mse_train': train_metrics['mse'],
                'rmse_train': train_metrics['rmse'],
                'r2_train': train_metrics['r2'],
                'mse_val': val_metrics['mse'],
                'rmse_val': val_metrics['rmse'],
                'r2_val': val_metrics['r2'],
                'mse_test': test_metrics['mse'],
                'rmse_test': test_metrics['rmse'],
                'r2_test': test_metrics['r2'],
                'gp_runtime_seconds': None,
                'eval_time_seconds': eval_time,
                'eval_memory_mb': eval_memory,
                'total_memory_mb': get_memory_usage()
            }
            save_results_to_csv(results_data, 'results/baseline_results.csv')


datasets = ['CARSALES', 'Electricity', 'GAS', 'LAKEERIE', 'Nordic', 'PIGS', 'POLLUTION', 'REDWINE', 'SUNSPOT', 'B1H']

logger.info(f"Starting baseline tests on {len(datasets)} datasets")
logger.info("="*60)

for dataset in tqdm(datasets, desc="Datasets", unit="dataset"):
    run_tests(dataset)

logger.info("\n" + "="*60)
logger.info("All baseline tests completed!")
logger.info("="*60)

Datasets:   0%|          | 0/10 [00:00<?, ?dataset/s]

Dataset: CARSALES
arima


/Users/lucasreis/Documents/RegressorGE/.venv/lib/python3.12/site-packages/sewar/utils.py:18: UserWarning: Supplied images have different dtypes float64 and float32
  warnings.warn(msg)


Métricas Treino - MSE: 1.351e-02, RMSE: 1.162e-01, R²: 0.634121
Métricas Validação - MSE: 1.882e-02, RMSE: 1.372e-01, R²: 0.461448
Métricas Teste - MSE: 1.704e-02, RMSE: 1.305e-01, R²: 0.517185

------------------



Dataset: CARSALES
arima + mlp


...

Métricas Treino - MSE: 9.644e-04, RMSE: 3.105e-02, R²: 0.975482
Métricas Validação - MSE: 2.867e-04, RMSE: 1.693e-02, R²: 0.988388
Métricas Teste - MSE: 5.250e-04, RMSE: 2.291e-02, R²: 0.983113

------------------

Dataset: B1H
sarima + svr


/Users/lucasreis/Documents/RegressorGE/.venv/lib/python3.12/site-packages/sewar/utils.py:18: UserWarning: Supplied images have different dtypes float64 and float32
  warnings.warn(msg)



Métricas Treino - MSE: 4.302e-04, RMSE: 2.074e-02, R²: 0.989062
Métricas Validação - MSE: 2.349e-04, RMSE: 1.533e-02, R²: 0.990484
Métricas Teste - MSE: 4.229e-04, RMSE: 2.056e-02, R²: 0.986398

------------------

Dataset: B1H
sarima + rbf


/Users/lucasreis/Documents/RegressorGE/.venv/lib/python3.12/site-packages/sewar/utils.py:18: UserWarning: Supplied images have different dtypes float64 and float32
  warnings.warn(msg)


Datasets: 100%|██████████| 10/10 [00:45<00:00,  4.53s/dataset]

Métricas Treino - MSE: 1.515e-03, RMSE: 3.892e-02, R²: 0.961489
Métricas Validação - MSE: 7.479e-04, RMSE: 2.735e-02, R²: 0.969707
Métricas Teste - MSE: 1.102e-03, RMSE: 3.319e-02, R²: 0.964568

------------------

